# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [24]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [25]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [26]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [27]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [28]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [29]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [30]:
gamma = 1
sigma = .005

In [31]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [32]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

Sólo puedes pedir las preguntas 1 vez.


In [33]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 2:

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 582 y 754 días.


Pregunta 3:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,022.00 (365 días después).


Pregunta 4:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .5504 .


Pregunta 5:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 6:

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo

## Respuestas

### Pregunta 1

**Enunciado**

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 98.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.

**Respuesta**

In [73]:
gamma = 1
sigma = 0.5e-2

value = hw.zcb_call_put(
            hw.CallPut.CALL,
            0.98,
            curva(0),
            1,
            2,
            math.exp(-curva(1*365) * 1),
            math.exp(-curva(2*365) * 2),
            gamma,
            sigma)
print(f'Valor Call {value:.8}')

Valor Call 0.019553576


### Pregunta 2

**Enunciado**

Con la curva cero cupón, interpolando linealmente en tasa, calcule la tasa
forward entre 582 y 754 días.

**Respuesta**

In [52]:
t1 = 582
t2 = 754
r1 = curva(t1)
r2 = curva(t2)
df1 = (1+r1)**(-t1/365)
df2 = (1+r2)**(-t2/365)
fwd_rate = (df1/df2-1)*360/(t2-t1)
fwd_rate

print(f'Tasa Forward {fwd_rate:.8%}')

Tasa Forward 0.04115688%


### Pregunta 3

**Enunciado**

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,022.00 (365 días después).

**Respuesta**

In [53]:
ICP_0 = 10e3
ICP_365 = 10022
d = 365

TNA = round(((ICP_365/ICP_0 - 1) * 360 / d), 4)

print(f'TNA {TNA:.4%}')

TNA 0.2200%


### Pregunta 4

**Enunciado**

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a .5504 .

**Respuesta**

Considerando la dinámica de la tasa:

$$dr_t=\gamma\left(\overline{r}-r_t\right)dt+\sigma dX^*_t$$

La discretización de Euler se lee como:

$$ r_1 = r_0 + \gamma\left(\overline{r}-r_0\right)dt + \sigma \sqrt{dt} \cdot Shock_1$$



In [77]:
r0 = curva(0)
r_ = 0.5e-2
dt = 1/264
gamma = 1
sigma = 0.5e-2
shock = 0.5504

r1 = r0 + gamma * (r_ - r0) * dt + sigma * math.sqrt(dt) * shock

print(f'La tasa en el paso 1 corresponde a {r1:.8%}')



La tasa en el paso 1 corresponde a 0.09914702%


### Pregunta 5

**Enunciado** 

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.

**Respuesta**

En cuanto al cálculo estocástico, lo único que cambia es el drift de la dinámica del precio del instrumento, que pasa de $m^*$ a $m^*\left(r_t,t\right)+\sigma_Z\left(r_t,t\right)\sigma\left(r_t,t\right)$, además de que el término $rV$ de la SDE desaparece. 

\begin{equation}
\frac{\partial\overline{V}}{\partial t}+\frac{\partial\overline{V}}{\partial r}\left[m^*\left(r_t,t\right)+\sigma_Z\left(r_t,t\right)\sigma\left(r_t,t\right)\right]+\frac{1}{2}\frac{\partial^2\overline{V}}{\partial r^2}\sigma^2\left(r_t,t\right)=0
\end{equation}

Por lo que reflejado en la simulación esto lleva a modificar la función $\theta_t$ del modelo de HW.

Ahora bien, la razón de porque resulta tan útil el cambio de medida es porque el factor de descuento $\displaystyle \exp \int r_s ds$ deja de ser estocástico, y se convierto en algo super conocido y observable que es el precio del bono cupón cero. Por lo que al momento de simular en la medida forward, no es necesario guardar las trayectoria de las tasas, porque el df es una constante. Computacionalmente esto es un plus. En caso de medida ajustada por riesgo, se debe guardar la trayectoria, porque se calcula un factor descuento para cada simulación que depende de la trayectoria de cada uno de los paths.


### Pregunta 6

**Enunciado**: 

La fórmula para una opción sobre un bono cupón cero en el modelo de Vasicek es
la misma que en el modelo de Hull-White. Comente.

**Respuesta**: 

Efectivamente, la fórmula para una opción, sea call o put, sobre un bono cupón cero es igual para los dos modelos. Si bien los modelos son distintos, en ambos caso se puede hacer uso de la conocida ecuación de Black-Scholes-Merton, que representa el valor presente del valor esperado de un payoff que depende de un subyacente, como lo es el precio del bono cupón cero, que distribuye lognormal.

Si bien estos modelos de tasas tienen dinámicas que dependen de parámetros distintos en cuanto al drift, el coeficiente de difusión es el mismo en ambos. Lo que lleva a que la ecuanción de BSM sea valida para los dos casos.


### Pregunta 7

**Enunciado**

Considere un swap a 1Y con cupones semestrales de 1.2% (lineal). Si la tasa
cupón cero a 0.5Y es 1.1% (exp), calcule la tasa cero a 1Y.

**Respuesta**

Primero se debe calcular el factor de descuento a 6M a partir de la tasa cupón cero a 6M.

In [88]:
r_6M = 1.1e-2
yf_6M = 0.5
yf_6M_1Y = 0.5
yf_1Y = 1
df6M = math.exp(-r_6M*yf_6M)

print(f'Factor de descuento 6M: {df6M:.4}')

Factor de descuento 6M: 0.9945


Suponiendo que la pata fija vale par se tiene que
$$r_{swap,1Y} \cdot yf_{0,6M} \cdot df_{6M} + (1 + r_{swap, 1Y} \cdot yf_{6M,1Y}) = 1$$
se puede despejar facilmente el factor de descuento a 1Y.

In [89]:
r_swap1Y = 1.2e-2
df1Y = (1 - r_swap1Y * yf_6M * df6M) / (1 + r_swap1Y * yf_6M_1Y)

print(f'Factor de descuento 1Y: {df1Y:.4}')

Factor de descuento 1Y: 0.9881


Finalmete, se determina la tasa a 1Y a partir del df a 1Y.

In [90]:
r_1Y = -math.log(df1Y)/yf_1Y

print(f'Tasa cero a 1Y {r_1Y:.4%}')

Tasa cero a 1Y 1.1967%
